In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(0,'../')

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV

from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
    
from common.featuresutils import load_features
from common.flagutils import load_flag_list
from common.flagutils import load_o3_flags
from common.flagutils import build_config
from common.flagutils import get_cmd_string_from_config

In [2]:
data = pd.read_csv('CE.results.csv')
data.loc[:, 'Energy'] *= 1e-6 # Convert energy to Joules
data.loc[:, 'Benchmark'] = data['Benchmark'].str.replace('.x', '')

average_data = data.groupby(['Benchmark','Flags', 'Type', 'RunId'], as_index=False).agg({'Energy':'mean', 'Time':'mean'})
average_data.head()

,Benchmark,Flags,Type,RunId,Energy,Time
0,BT.C,-O3,O3,-1,8685.049255,44.226147
1,BT.C,-O3 -faggressive-loop-optimizations -falign-fu...,test,80,8776.150006,43.287949
2,BT.C,-O3 -faggressive-loop-optimizations -falign-fu...,test,82,8918.038919,43.911705
3,BT.C,-O3 -faggressive-loop-optimizations -falign-fu...,test,83,8861.715663,43.573321
4,BT.C,-O3 -faggressive-loop-optimizations -falign-fu...,test,84,8867.013709,43.667829


In [3]:
all_flags = load_flag_list()
print(len(all_flags))

187


In [4]:
def best_configuration(variable, benchmark, average_data):
    benchmark_data = average_data.loc[average_data["Benchmark"] == benchmark]
    min_index = benchmark_data[variable].idxmin()
    return benchmark_data.loc[min_index]["Flags"]


def get_o3_config():
    return get_cmd_string_from_config(build_config(all_flags, load_o3_flags(), '-O3'))

In [5]:
benchmarks = data["Benchmark"].unique()
benchmarks_nosize = [b.split('.')[0] for b in benchmarks]
print(benchmarks_nosize)

['nab', 'imagick', 'botsalgn', 'botsspar', 'kdtree', 'smithwa', 'BT', 'UA', 'swim', 'EP']


In [6]:
def load_data(with_dwarf):
    X = StandardScaler().fit_transform(load_features(benchmarks_nosize, with_dwarf=with_dwarf))
    y = []

    for benchmark in benchmarks:
        config_str = best_configuration('Energy', benchmark, average_data)
        if config_str == '-O3':
            config_str = get_o3_config()

        labels = []

        config = config_str[4:].split(' ')
        for i, flag in enumerate(config):
            if flag == all_flags[i]:
                labels.append(1) # Flag is turned on
            elif flag == '-fno-' + all_flags[i][2:]:
                labels.append(0) # FLag is turned off -fno
            else:
                print("ERROR:" + flag)

        y.append(labels)

    return X, np.array(y)


def remove_static_labels(y):
    '''Removes any columns that are the same for all samples'''
    cols_to_remove = []
    for i in range(y.shape[1]):
        unique, counts = np.unique(y[:, i], return_counts=True)

        if (len(unique) == 1) or (1 in counts):
            cols_to_remove.append(i)

    return np.delete(y, cols_to_remove, axis=1)


def grid_search(classifier, parameters, X, y):
    gs = GridSearchCV(classifier, parameters, cv=10, scoring='f1_macro', n_jobs=-1)
    gs.fit(X, y)

    return gs


def test_multilabel_classifier(classifier, X, y):
    classifier_name = type(classifier).__name__
    
    print(f"Testing {classifier_name}")
    
    parameters = [
        {
            'classifier': [SVC(kernel='linear')],
            'classifier__C': np.logspace(-2, 10, 13)
        },
        {
            'classifier': [SVC(kernel='rbf')],
            'classifier__C': np.logspace(-2, 10, 13), 
            'classifier__gamma':  np.logspace(-9, 3, 13)
        },
        {
            'classifier': [GaussianNB(), DecisionTreeClassifier(max_depth=5), AdaBoostClassifier(), DecisionTreeClassifier(max_depth=5)]
        }
    ]
    gs = grid_search(classifier, parameters, X, y)

    print(f"{classifier_name} Best F1 Score: {gs.best_score_}")
    print(gs.best_params_)
    print("---------")    

In [7]:
def run_comparison(multilabel_classifiers, disable_warnings=False, with_dwarf=False):
    X, y = load_data(with_dwarf)
    y = remove_static_labels(y)

    print(f"Loaded {X.shape[0]} samples with {X.shape[1]} features and {y.shape[1]} labels.")
    
    # Grid search can throw a lot of warnings. Is useful to just mute them sometimes.
    if disable_warnings:
        import warnings
        warnings.filterwarnings('ignore')
        
    for clf in multilabel_classifiers:
        test_multilabel_classifier(clf, X, y)

In [8]:
run_comparison([BinaryRelevance(), ClassifierChain(), LabelPowerset()], disable_warnings=True, with_dwarf=False)

Loaded 10 samples with 65 features and 85 labels.
Testing BinaryRelevance
BinaryRelevance Best F1 Score: 0.6517647058823529
{'classifier': SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1e-09, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False), 'classifier__C': 0.01, 'classifier__gamma': 1e-09}
---------
Testing ClassifierChain
ClassifierChain Best F1 Score: 0.6529411764705882
{'classifier': SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1.0, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False), 'classifier__C': 1.0, 'classifier__gamma': 1.0}
---------
Testing LabelPowerset
LabelPowerset Best F1 Score: 0.6047058823529412
{'classifier': SVC(C=100000000.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1e-09

In [9]:
run_comparison([BinaryRelevance(), ClassifierChain(), LabelPowerset()], disable_warnings=True, with_dwarf=True)

Loaded 10 samples with 66 features and 85 labels.
Testing BinaryRelevance
BinaryRelevance Best F1 Score: 0.6517647058823529
{'classifier': SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1e-09, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False), 'classifier__C': 0.01, 'classifier__gamma': 1e-09}
---------
Testing ClassifierChain
ClassifierChain Best F1 Score: 0.6529411764705882
{'classifier': SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False), 'classifier__C': 0.01}
---------
Testing LabelPowerset
LabelPowerset Best F1 Score: 0.4705882352941176
{'classifier': SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_